In [1]:
import pandas as pd
from numpy import random
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import numpy as np

from scripts.scrapper_gallito import flatten, scrapper, scrap_latlng, format_barrio, encode_barrio

In [2]:
base_url = 'https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={}'
all_urls= [base_url.format(i) for i in range(1, 200)]
all_urls[0:3]


['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag=1',
 'https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag=2',
 'https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag=3']

### Scrapper multi-hilo

In [3]:
threads = 15

urls = np.array_split(all_urls, threads)

all_r = []

with ThreadPoolExecutor(threads) as executor:
    for result in executor.map(scrapper, urls):
        all_r.append(result)

In [12]:
data = pd.concat(all_r).reset_index(drop=True)
data.head()

,desc,valor,metraje,url
0,3 Dormitorios en Aires Puros,$U 190.000,110,https://www.gallito.com.uy/excelente-garaje-2-...
1,2 Dormitorios en Prado,U$S 12.900,68,https://www.gallito.com.uy/a-estrenar-sin-gc-d...
2,3 Dormitorios en Prado,U$S 15.500,100,https://www.gallito.com.uy/gran-oportunidad-in...
3,3 Dormitorios en Villa Española,U$S 38.000,150,https://www.gallito.com.uy/casa-3-dormitorios-...
4,2 Dormitorios en La Teja,U$S 39.900,60,https://www.gallito.com.uy/economica-con-terre...


In [13]:
data.shape

(5360, 4)

In [14]:
# limpieza del campo valor
data['valor'].str.replace('U', '')

for word, rep in {"U":" ", "S":"", "$":"", ".":""}.items():
    data['valor'] = data['valor'].str.replace(word, rep, regex=False)

data['valor'] = data['valor'].astype(int)

# elimina outliers en valor
data = data.loc[~((data.valor == 111111111) | (data.valor < 30000))]

# depura campo metraje
data['metraje'] = data['metraje'].str.strip()
data.loc[data['metraje'] == '', 'metraje'] = np.nan

# elimina outliers en metraje
data['metraje'] = data['metraje'].astype(float)

data = data.loc[~((data.metraje > 2000) | (data.metraje < 15) | (data.metraje.isna()))]

In [15]:
data.metraje.describe()

count    4304.000000
mean      173.317612
std       132.487442
min        20.000000
25%        88.000000
50%       137.000000
75%       210.000000
max      1599.000000
Name: metraje, dtype: float64

In [14]:
data.valor.describe().apply(lambda x: format(x, 'f'))

count       4304.000000
mean      261922.592240
std       182763.840163
min        30000.000000
25%       130000.000000
50%       199000.000000
75%       349999.250000
max      1500000.000000
Name: valor, dtype: object

In [17]:
data.loc[data.metraje == 20]

,desc,valor,metraje,url
764,1 Ambiente en Cerro,31000,20.0,https://www.gallito.com.uy/venta-apartamento-m...


In [18]:
fecha_str = str(datetime.today().date())
filename = f'data/ventas_mvdeo_{fecha_str}.csv'
data.to_csv(filename, index=False)

## Recupera coordenadas

In [2]:
filename = 'data/ventas_mvdeo_2022-08-04.csv'
data = pd.read_csv(filename)
print(data.shape)
data.head()

(4304, 4)


,desc,valor,metraje,url
0,3 Dormitorios en Aires Puros,190000,110.0,https://www.gallito.com.uy/excelente-garaje-2-...
1,3 Dormitorios en Villa Española,38000,150.0,https://www.gallito.com.uy/casa-3-dormitorios-...
2,2 Dormitorios en La Teja,39900,60.0,https://www.gallito.com.uy/economica-con-terre...
3,4 Dormitorios en Cerrito,45000,97.0,https://www.gallito.com.uy/oportunidad-casa-de...
4,2 Dormitorios en Colon,45000,50.0,https://www.gallito.com.uy/oficina-sosa-proxim...


In [3]:
data.tail()

,desc,valor,metraje,url
4299,Más de 4 dormitorios en Parque Rodo,790000,550.0,https://www.gallito.com.uy/se-vende-edificio-e...
4300,4 Dormitorios en Carrasco,790000,439.0,https://www.gallito.com.uy/venta-casa-carrasco...
4301,Más de 4 dormitorios en Carrasco,790000,439.0,https://www.gallito.com.uy/casa-carrasco-4-dor...
4302,4 Dormitorios en Prado,790000,616.0,https://www.gallito.com.uy/venta-casa-4-dormit...
4303,4 Dormitorios en Carrasco,790000,320.0,https://www.gallito.com.uy/oportunidad-en-el-c...


In [4]:
# multithreaded scrapping
threads = 10

urls = np.array_split(list(data.url.values), threads)

len(urls)

all_r = []

with ThreadPoolExecutor(threads) as executor:
    for result in executor.map(scrap_latlng, urls):
        all_r.append(result)

Message: unknown error: missing or invalid columnNumber
  (Session info: headless chrome=104.0.5112.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x008778B3+2193587]
	Ordinal0 [0x00810681+1771137]
	Ordinal0 [0x007241A8+803240]
	Ordinal0 [0x007120FA+729338]
	Ordinal0 [0x0071163B+726587]
	Ordinal0 [0x00717704+751364]
	Ordinal0 [0x00717C25+752677]
	Ordinal0 [0x00717A0F+752143]
	Ordinal0 [0x00716FAF+749487]
	Ordinal0 [0x00715E94+745108]
	Ordinal0 [0x00716498+746648]
	Ordinal0 [0x0072CF37+839479]
	Ordinal0 [0x007266BB+812731]
	Ordinal0 [0x0072620D+811533]
	Ordinal0 [0x00726A2F+813615]
	Ordinal0 [0x00726D00+814336]
	Ordinal0 [0x007521B5+991669]
	Ordinal0 [0x0075273B+993083]
	Ordinal0 [0x0077F7FC+1177596]
	Ordinal0 [0x0076D7F4+1103860]
	Ordinal0 [0x0077DAE2+1170146]
	Ordinal0 [0x0076D5C6+1103302]
	Ordinal0 [0x007477E0+948192]
	Ordinal0 [0x007486E6+952038]
	GetHandleVerifier [0x00B20CB2+2738370]
	GetHandleVerifier [0x00B121B8+2678216]
	GetHandleVerifier [0x009017AA+512954]
	GetHandleVerifier [0x009008

In [6]:
coords = flatten(all_r)

In [7]:
len(coords)

4304

### Limpia de dataframe y guarda las coordenadas

In [ ]:
data = pd.read_csv(filename)

data.head()

In [8]:
r = [i=='Nan,Nan' for i in coords]
print(sum(r))

coords_splitted = [i.split(',') for i in coords]

1115


In [43]:
data[['lat', 'lng']] = coords_splitted

data['lat'] = data['lat'].astype(float).round(5)
data['lng'] = data['lng'].astype(float).round(5)

# depuras algunas coordenadas erróneas
data.loc[data.lat > -33 , 'lat'] = np.nan
data.loc[data.lng > -55, 'lng'] = np.nan

data.head(8)

,desc,valor,metraje,url,lat,lng,dormitorios,barrio,barrio_ine,cod_barrio_ine,valor_metro
0,3 Dormitorios en Aires Puros,190000,110.0,https://www.gallito.com.uy/excelente-garaje-2-...,NaN,NaN,3 Dormitorios,Aires Puros,Aires Puros,29.0,1727.272727
1,3 Dormitorios en Villa Española,38000,150.0,https://www.gallito.com.uy/casa-3-dormitorios-...,-34.86785,-56.14285,3 Dormitorios,Villa Española,Villa Española,24.0,253.333333
2,2 Dormitorios en La Teja,39900,60.0,https://www.gallito.com.uy/economica-con-terre...,-34.85720,-56.24105,2 Dormitorios,La Teja,La Teja,38.0,665.000000
3,4 Dormitorios en Cerrito,45000,97.0,https://www.gallito.com.uy/oportunidad-casa-de...,-34.85944,-56.15642,4 Dormitorios,Cerrito,Cerrito,27.0,463.917526
4,2 Dormitorios en Colon,45000,50.0,https://www.gallito.com.uy/oficina-sosa-proxim...,-34.80505,-56.23007,2 Dormitorios,Colon,"Colón Sureste, Abayubá",58.0,900.000000
5,2 Dormitorios en J. Hipodromo,50000,70.0,https://www.gallito.com.uy/casa-en-venta-jardi...,-34.83717,-56.15027,2 Dormitorios,J. Hipodromo,Jardines del Hipódromo,21.0,714.285714
6,2 Dormitorios en Union,50000,50.0,https://www.gallito.com.uy/gran-oportunidad-ve...,NaN,NaN,2 Dormitorios,Union,Unión,23.0,1000.000000
7,1 Dormitorio en Cerrito,53000,33.0,https://www.gallito.com.uy/baldovino-cerrito-t...,NaN,NaN,1 Dormitorio,Cerrito,Cerrito,27.0,1606.060606


In [44]:
data[['dormitorios', 'barrio']] = data['desc'].str.split(' en ', n = 1, expand = True)

In [45]:
data['barrio_ine'] = data['barrio']

In [46]:
data['barrio_ine'] = format_barrio(data, 'barrio')

In [47]:
data.loc[:, 'cod_barrio_ine'] = encode_barrio(data, 'barrio_ine')

In [48]:
data.to_csv(f'data/ventas_mvdeo_{fecha_str}_depurado.csv', index=False)

In [49]:
# calcula valor por metro
data['valor_metro'] = data['valor'] / data['metraje']

# agrupa por barrio, calculando la mediana del valor por metro
agru = data.groupby('cod_barrio_ine').median('valor_metro').reset_index()

# cambia tipo de campo del barrio INE
agru['cod_barrio_ine'] = agru['cod_barrio_ine'].astype(int)

# guarda en CSV
agru.to_csv(f'data/datos_agrupados_{fecha_str}.csv', index=False)